<a href="https://colab.research.google.com/github/ewertonUrso/99/blob/main/loft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

##############################
# PRODUCT SALES
##############################

##############################
# 1. Importing data about STORES
f = '/content/drive/MyDrive/Loft/lojas.csv'
stores = pd.read_csv(f)
stores = stores.rename(columns = {'Loja' : 'store',
                                  'Identificador' : 'store_id'})
stores['store'] = stores['store'].astype('string')
stores['store_id'] = stores['store_id'].astype('string')

##############################
# 2. Importing data about PRODUCTS
f = '/content/drive/MyDrive/Loft/produtos.csv'
products = pd.read_csv(f)
products = products.rename(columns = {'Produto' : 'product',
                                      'Id_Produto' : 'product_id',
                                      'Valor Unitário' : 'unit_value'})
products['product'] = products['product'].astype('string')
products['product_id'] = products['product_id'].astype('string')
products['unit_value'] = products['unit_value'].astype('float')

##############################
# 3. Importing data about COSTS
f = '/content/drive/MyDrive/Loft/custos.csv'
costs = pd.read_csv(f)
costs = costs.rename(columns = {'Data' : 'date',
                                'Loja_Id' : 'store_id',
                                'Custos' : 'costs'})
costs['date'] = pd.to_datetime(costs['date'])
costs['store_id'] = costs['store_id'].astype('string')
costs['costs'] = costs['costs'].astype('float')

##############################
# 4. Importing data about SALES
f = '/content/drive/MyDrive/Loft/vendas.csv'
sales = pd.read_csv(f)
sales = sales.rename(columns = {'Data' : 'date',
                                'Identificador Produto' : 'product_id',
                                'Unidades Vendidas' : 'sold_unities',
                                'Loja_Id' : 'store_id',
                                'Valor Unitário' : 'unit_value'})
sales['date'] = pd.to_datetime(sales['date'])
sales['sale_day'] = sales['date'].dt.day.astype('int')
sales['sale_month'] = sales['date'].dt.month.astype('int')
sales['sale_year'] = sales['date'].dt.year.astype('int')
sales['sale_month_year'] = sales['date'].dt.month.astype('string') + '-' + sales['date'].dt.year.astype('string')
sales['product_id'] = sales['product_id'].astype('string')
sales['sold_unities'] = sales['sold_unities'].astype('int')
sales['store_id'] = sales['store_id'].astype('string')
sales['unit_value'] = sales['unit_value'].astype('float')

##############################
# Merging datasets
sales_products = sales.merge(stores, on = 'store_id') \
                      .merge(products, on = 'product_id', suffixes = ('', '_'))
sales_products['sale_id'] = sales_products.index + 1
sales_products = sales_products[['sale_id',
                                 'date', 'sale_day', 'sale_month', 'sale_year', 'sale_month_year',
                                 'store_id', 'store',
                                 'product_id', 'product', 'unit_value',
                                 'sold_unities']]

##############################
# Exporting data
f = '/content/drive/MyDrive/Loft/loft_data.csv'
sales_products.to_csv(path_or_buf = f, sep = ',', index = False)

##############################
# A new dataset with different column names
sales_products_ref = sales_products[['date', 'sale_month',
                                     'store',
                                     'product',
                                     'unit_value',
                                     'sold_unities']]

sales_products_ref = sales_products_ref.rename(columns = {'store' : 'store_ref',
                                                          'product' : 'product_ref'})
f = '/content/drive/MyDrive/Loft/loft_data_fixed.csv'
sales_products_ref.to_csv(path_or_buf = f, sep = ',', index = False)

##############################
# FINANCIAL RESULTS
##############################

# Revenues
revenues = sales_products[['date',
                           'store_id', 'store',
                           'unit_value', 'sold_unities']]
revenues['sale_value'] = revenues['unit_value'] * revenues['sold_unities']
revenues = revenues[['date',
                     'store_id', 'store',
                     'sale_value']]
revenues = revenues.groupby(['date',
                             'store_id', 'store'], as_index = False).agg('sum')
revenues = revenues[['date', 'store', 'sale_value']]
revenues = revenues.rename(columns = {'sale_value' : 'value'})
revenues['value_class'] = 'revenue'
revenues['id'] = revenues.index
revenues = revenues[['id', 'date', 'store', 'value', 'value_class']]

# Costs
costs = costs.groupby(['date', 'store_id'], as_index = False).agg('sum')
stores = sales_products[['store_id', 'store', 'sold_unities']] \
          .groupby(['store_id', 'store'], as_index = False) \
          .agg('sum')[['store_id', 'store']]
costs = costs.merge(stores, on = 'store_id')
costs = costs[['date', 'store', 'costs']]
costs = costs.rename(columns = {'costs' : 'value'})
costs['value_class'] = 'cost'
costs['id'] = costs.index
costs = costs[['id', 'date', 'store', 'value', 'value_class']]

financial_results = pd.concat([revenues, costs])
frp = pd.pivot(financial_results, index = ['id', 'date', 'store'], columns = 'value_class', values = 'value')
frp = frp.reset_index()

# Exporting data
f = '/content/drive/MyDrive/Loft/financial_results.csv'
frp.to_csv(path_or_buf = f, sep = ',', index = False)
